In [28]:
#taken from stanford tensorflow tutorial for cnn
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time 

import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
#train data class normal 38 class bad 280 
#test data class normal 10 class bad 20
import utils
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
DATASET_PATH = 'E:\HolisticClassification\Dataset' # the dataset file or root folder path.
TEST_DATASET_PATH = 'E:\HolisticClassification\Dataset\Test'
BATCH_SIZE = 16

# Image Parameters
N_CLASSES = 2 # CHANGE HERE, total number of classes
IMG_HEIGHT = 32 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 32 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale
def read_images(dataset_path = DATASET_PATH, mode = MODE, batch_size = BATCH_SIZE):
    imagepaths, labels = list(), list()
    if mode == 'folder':
        label = 0
        classes = [f.path for f in os.scandir(DATASET_PATH) if f.is_dir() ] 
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.bmp') or sample.endswith('.BMP'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")
    resized_image_array=[]

    for image_loc in imagepaths:
        image_decoded = tf.image.decode_bmp(tf.gfile.FastGFile(image_loc, 'rb').read(),channels=CHANNELS)
        resized_image = tf.reshape(tf.image.resize_images(image_decoded, [IMG_HEIGHT,IMG_HEIGHT]),[1,IMG_HEIGHT*IMG_HEIGHT*CHANNELS])
        resized_image_array.append(resized_image)
    resized_image_array = tf.reshape(tf.stack(resized_image_array),[len(imagepaths),IMG_HEIGHT*IMG_HEIGHT*CHANNELS])
    # Normalize
    #image = image * 1.0/127.5 - 1.0
    image = tf.convert_to_tensor(resized_image_array,dtype=tf.float32)
    labels = tf.convert_to_tensor(labels,dtype=tf.float32);
    ilabels = tf.reshape(labels,[len(imagepaths),1])
    train_data = tf.concat([image, ilabels], 1)
    train_data = tf.data.Dataset.from_tensor_slices(train_data)
    train_data = train_data.shuffle(128)
    train_data = train_data.batch(BATCH_SIZE)

    # Create batches
   # X, Y = tf.train.batch([image, label], batch_size=batch_size, capacity=batch_size * 8, num_threads=4)
    return train_data

def read_test_images(dataset_path = TEST_DATASET_PATH, mode = MODE, batch_size = BATCH_SIZE):
    imagepaths =  list()
    if mode == 'folder':
        classes = [f.path for f in os.scandir(DATASET_PATH) if f.is_dir() ] 
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.bmp') or sample.endswith('.BMP'):
                    imagepaths.append(os.path.join(c_dir, sample))
    else:
        raise Exception("Unknown mode.")

    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    # Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer(imagepaths,
                                                 shuffle=True)

    # Read images from disk
    
    image = tf.image.decode_bmp(image, channels=CHANNELS)

    # Resize images to a common size
    image = tf.image.resize_images(image, [IMG_HEIGHT, IMG_WIDTH])

    # Normalize
    #image = image * 1.0/127.5 - 1.0
    test = image
    test_data = tf.data.Dataset.from_tensor_slices(test)
    test_data = test_data.batch(BATCH_SIZE)
    # Create batches
    # Create batches
   # X, Y = tf.trai.batch([image, label], batch_size=batch_size, capacity=batch_size * 8, num_threads=4)
    return test_data


class ConvNet(object):
    def __init__(self):
        self.lr = 0.001
        self.batch_size = BATCH_SIZE
        self.keep_prob = tf.constant(0.75)
        self.gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')
        self.n_classes = 2
        self.skip_step = 20
        self.n_test = 30
        self.training=False

    def get_data(self):
        with tf.name_scope('data'):
            train_data = read_images()
            test_data = read_test_images()
            iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                                   train_data.output_shapes)
            img, self.label = iterator.get_next()
            self.img = tf.reshape(img, shape=[-1, IMG_WIDTH, IMG_HEIGHT, CHANNELS])
            # reshape the image to make it work with tf.nn.conv2d

            self.train_init = iterator.make_initializer(train_data)  # initializer for train_data
            self.test_init = iterator.make_initializer(test_data)    # initializer for train_data

    def inference(self):
        conv1 = tf.layers.conv2d(inputs=self.img,
                                  filters=32,
                                  kernel_size=[5, 5],
                                  padding='SAME',
                                  activation=tf.nn.relu,
                                  name='conv1')
        pool1 = tf.layers.max_pooling2d(inputs=conv1, 
                                        pool_size=[2, 2], 
                                        strides=2,
                                        name='pool1')

        conv2 = tf.layers.conv2d(inputs=pool1,
                                  filters=64,
                                  kernel_size=[5, 5],
                                  padding='SAME',
                                  activation=tf.nn.relu,
                                  name='conv2')
        pool2 = tf.layers.max_pooling2d(inputs=conv2, 
                                        pool_size=[2, 2], 
                                        strides=2,
                                        name='pool2')

        feature_dim = pool2.shape[1] * pool2.shape[2] * pool2.shape[3]
        pool2 = tf.reshape(pool2, [-1, feature_dim])
        fc = tf.layers.dense(pool2, 128, activation=tf.nn.relu, name='fc')
        dropout = tf.layers.dropout(fc, 
                                    self.keep_prob, 
                                    training=self.training, 
                                    name='dropout')
        self.logits = tf.layers.dense(dropout, self.n_classes, name='logits')

    def loss(self):
        '''
        define loss function
        use softmax cross entropy with logits as the loss function
        compute mean cross entropy, softmax is applied internally
        '''
        # 
        with tf.name_scope('loss'):
            entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.label, logits=self.logits)
            self.loss = tf.reduce_mean(entropy, name='loss')
    
    def optimize(self):
        '''
        Define training op
        using Adam Gradient Descent to minimize cost
        '''
        self.opt = tf.train.AdamOptimizer(self.lr).minimize(self.loss, 
                                                global_step=self.gstep)

    def summary(self):
        '''
        Create summaries to write on TensorBoard
        '''
        with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.accuracy)
            tf.summary.histogram('histogram loss', self.loss)
            self.summary_op = tf.summary.merge_all()
    
    def eval(self):
        '''
        Count the number of right predictions in a batch
        '''
        with tf.name_scope('predict'):
            preds = tf.nn.softmax(self.logits)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.label, 1))
            self.accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

    def build(self):
        '''
        Build the computation graph
        '''
        self.get_data()
        self.inference()
        self.loss()
        self.optimize()
        self.eval()
        self.summary()

    def train_one_epoch(self, sess, saver, init, writer, epoch, step):
        start_time = time.time()
        sess.run(init) 
        self.training = True
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l, summaries = sess.run([self.opt, self.loss, self.summary_op])
                writer.add_summary(summaries, global_step=step)
                if (step + 1) % self.skip_step == 0:
                    print('Loss at step {0}: {1}'.format(step, l))
                step += 1
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        saver.save(sess, 'checkpoints/convnet_layers/mnist-convnet', step)
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/n_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step

    def eval_once(self, sess, init, writer, epoch, step):
        start_time = time.time()
        sess.run(init)
        self.training = False
        total_correct_preds = 0
        try:
            while True:
                accuracy_batch, summaries = sess.run([self.accuracy, self.summary_op])
                writer.add_summary(summaries, global_step=step)
                total_correct_preds += accuracy_batch
        except tf.errors.OutOfRangeError:
            pass

        print('Accuracy at epoch {0}: {1} '.format(epoch, total_correct_preds/self.n_test))
        print('Took: {0} seconds'.format(time.time() - start_time))

    def train(self, n_epochs):
        '''
        The train function alternates between training one epoch and evaluating
        '''
        utils.safe_mkdir('checkpoints')
        utils.safe_mkdir('checkpoints/convnet_layers')
        writer = tf.summary.FileWriter('./graphs/convnet_layers', tf.get_default_graph())

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_layers/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
            
            step = self.gstep.eval()

            for epoch in range(n_epochs):
                step = self.train_one_epoch(sess, saver, self.train_init, writer, epoch, step)
                self.eval_once(sess, self.test_init, writer, epoch, step)
        writer.close()

if __name__ == '__main__':
    model = ConvNet()
    model.build()
    model.train(n_epochs=15)   

NameError: name 'labels' is not defined